In [1]:
# Cell 1: 导入必要的包
import torch
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np
from ppq.api import espdl_quantize_onnx, get_target_platform
from ppq.core import TargetPlatform
from ppq.api.setting import QuantizationSettingFactory


    ___________ ____        ____  ____  ____ 
   / ____/ ___// __ \      / __ \/ __ \/ __ \
  / __/  \__ \/ /_/ /_____/ /_/ / /_/ / / / /
 / /___ ___/ / ____/_____/ ____/ ____/ /_/ / 
/_____//____/_/         /_/   /_/    \___\_\ 




In [2]:
# Cell 2: 准备校准数据集
class CalibrationDataset(Dataset):
    def __init__(self, X):
        self.X = torch.FloatTensor(X).unsqueeze(1)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx]

# 加载校准数据
with open('../dataset/cal.pkl', 'rb') as f:
    X_cal, _ = pickle.load(f)

cal_dataset = CalibrationDataset(X_cal)
cal_loader = DataLoader(cal_dataset, batch_size=32, shuffle=False)

In [3]:
# Cell 3: 配置量化参数并执行量化
ONNX_MODEL_PATH = "../training/gesture_model.onnx"
EXPORT_PATH = "../deployment/model/gesture_model_mixed.espdl"
input_shape = [1, 1, 96, 96]

# 创建量化设置
setting = QuantizationSettingFactory.espdl_setting()

# 配置混合精度量化
for layer in ["/first/first.0/Conv", "/first/first.2/Clip"]:
    setting.dispatching_table.append(
        layer, 
        get_target_platform("esp32s3", 16)  # 使用get_target_platform替代直接指定平台
    )

# 执行量化，参数遵循interface定义
graph = espdl_quantize_onnx(
    onnx_import_file=ONNX_MODEL_PATH,
    espdl_export_file=EXPORT_PATH,
    calib_dataloader=cal_loader,
    calib_steps=50,
    input_shape=[input_shape],
    target="esp32s3",
    num_of_bits=8,
    setting=setting,
    device="cpu",
    error_report=True,
    verbose=1
)

[14:33:58] PPQ Quantization Fusion Pass Running ...       Finished.
[14:33:58] PPQ Quantize Simplify Pass Running ...         Finished.
[14:33:58] PPQ Parameter Quantization Pass Running ...    Finished.
[14:33:58] PPQ Runtime Calibration Pass Running ...       

Calibration Progress(Phase 2): 100%|████████████| 50/50 [00:04<00:00, 10.01it/s]


Finished.
[14:34:07] PPQ Quantization Alignment Pass Running ...    Finished.
[14:34:07] PPQ Passive Parameter Quantization Running ... Finished.
--------- Network Snapshot ---------
Num of Op:                    [27]
Num of Quantized Op:          [27]
Num of Variable:              [74]
Num of Quantized Var:         [74]
------- Quantization Snapshot ------
Num of Quant Config:          [102]
ACTIVATED:                    [31]
OVERLAPPED:                   [33]
PASSIVE:                      [38]
Network Quantization Finished.


Analysing Graphwise Quantization Error(Phrase 1):: 100%|█| 8/8 [00:00<00:00, 13.
Analysing Graphwise Quantization Error(Phrase 2):: 100%|█| 8/8 [00:01<00:00,  5.


Layer                              | NOISE:SIGNAL POWER RATIO 
/layers/layers.1/conv/conv.6/Conv: | ████████████████████ | 347.308%
/layers/layers.2/conv/conv.3/Conv: | ███████████████████  | 327.313%
/layers/layers.2/conv/conv.0/Conv: | ███████████████████  | 326.920%
/classifier/classifier.2/Gemm:     | ████████████████     | 287.401%
/layers/layers.2/conv/conv.6/Conv: | ███████████████      | 269.612%
/layers/layers.3/conv/conv.3/Conv: | ███████████████      | 268.106%
/layers/layers.3/conv/conv.0/Conv: | ██████████████       | 246.990%
/layers/layers.1/conv/conv.3/Conv: | ██████████████       | 242.841%
/layers/layers.0/conv/conv.6/Conv: | █████████████        | 234.783%
/layers/layers.1/conv/conv.0/Conv: | ████████████         | 209.285%
/layers/layers.0/conv/conv.3/Conv: | █████████            | 166.581%
/layers/layers.3/conv/conv.6/Conv: | ████████             | 159.707%
/layers/layers.0/conv/conv.0/Conv: | ██                   | 61.218%
/first/first.0/Conv:               |     

Analysing Layerwise quantization error:: 100%|██| 14/14 [00:10<00:00,  1.36it/s]

Layer                              | NOISE:SIGNAL POWER RATIO 
/first/first.0/Conv:               | ████████████████████ | 296.322%
/layers/layers.1/conv/conv.3/Conv: |                      | 0.394%
/layers/layers.3/conv/conv.3/Conv: |                      | 0.292%
/layers/layers.0/conv/conv.3/Conv: |                      | 0.140%
/layers/layers.2/conv/conv.0/Conv: |                      | 0.131%
/layers/layers.2/conv/conv.3/Conv: |                      | 0.112%
/layers/layers.0/conv/conv.0/Conv: |                      | 0.108%
/layers/layers.3/conv/conv.0/Conv: |                      | 0.087%
/layers/layers.0/conv/conv.6/Conv: |                      | 0.061%
/layers/layers.1/conv/conv.6/Conv: |                      | 0.049%
/layers/layers.2/conv/conv.6/Conv: |                      | 0.048%
/layers/layers.1/conv/conv.0/Conv: |                      | 0.022%
/classifier/classifier.2/Gemm:     |                      | 0.018%
/layers/layers.3/conv/conv.6/Conv: |                      | 0.00

In [ ]:
# Cell 4: 评估量化后的模型
from ppq.executor import TorchExecutor
import time
from tqdm import tqdm

def evaluate_quantized_model(graph, test_loader, y_test):
    """
    评估量化后的模型性能
    """
    executor = TorchExecutor(graph=graph, device='cpu')
    total_time = 0
    correct = 0
    total = 0
    
    print("\n开始评估量化模型性能...")
    with torch.no_grad():
        for batch in tqdm(test_loader):
            start = time.time()
            outputs = executor.forward(inputs=batch)
            total_time += (time.time() - start)
            
            # 计算准确率
            _, predicted = torch.max(outputs[0], 1)
            total += batch.size(0)
            correct += (predicted == y_test[total-batch.size(0):total]).sum().item()

    # 计算并打印结果
    avg_time = (total_time / len(test_loader)) * 1000  # 转换为毫秒
    accuracy = (correct / total) * 100

    print(f"\n评估结果:")
    print(f"平均推理时间: {avg_time:.2f} ms")
    print(f"模型准确率: {accuracy:.2f}%")
    
    return avg_time, accuracy

# 加载测试数据
with open('../dataset/test.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

# 准备测试数据集
test_dataset = CalibrationDataset(X_test)  # 复用之前的Dataset类
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 执行评估
evaluate_quantized_model(graph, test_loader, y_test)

In [ ]:
!ls ../dataset

In [ ]:
# Cell 5: 使用量化后的模型识别一张图片
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_image(image_path):
    """
    图片预处理函数
    1. 读取图片
    2. 转换为灰度图
    3. 调整大小
    4. 归一化
    """
    # 读取图片
    img = cv2.imread(image_path)
    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 调整大小
    TARGET_SIZE = (96, 96)
    resized = cv2.resize(gray, TARGET_SIZE)
    # 归一化
    normalized = resized.astype('float32') / 255.0
    # 添加 batch 和 channel 维度
    image_tensor = torch.FloatTensor(normalized).unsqueeze(0).unsqueeze(0)
    
    return image_tensor, img

def predict_image(graph, image_tensor):
    """
    使用量化后的模型预测图像的类别置信度。
    """
    executor = TorchExecutor(graph=graph, device='cpu')
    with torch.no_grad():
        outputs = executor.forward(inputs=[image_tensor])
        confidences = torch.softmax(outputs[0], dim=1).numpy().flatten()
    
    return confidences

# 定义手势类别
gesture_labels = {
    '00': 'palm',
    '01': 'l',
    '02': 'fist',
    '03': 'thumb',
    '04': 'index',
    '05': 'ok',
    '06': 'c',
    '07': 'down'
}

# 加载并预处理图像
# 请将 "your_image.jpg" 替换为你要预测的图像的实际路径，例如："/path/to/your_image.jpg"
image_path = "../dataset/Dataset_Sample_Images_ByCategory/frame_00_03_0038.png"  # 这里将 "your_image.jpg" 替换为你要使用的图像路径
image_tensor, original_image = preprocess_image(image_path)

# 使用量化后的模型进行预测
confidences = predict_image(graph, image_tensor)

# 打印模型输出的各个置信度，并关联类别标签
print("\n预测结果: \n")
for idx, confidence in enumerate(confidences):
    label = gesture_labels[f'{idx:02}']
    print(f"类别: {label}, 置信度: {confidence:.4f}")

# 显示原始图片
plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')
plt.show()